In [12]:
import os
import cv2
import yaml

from tqdm import tqdm

In [2]:
# Load class names from data.yaml file
def load_classes_from_yaml(yaml_path):
    with open(yaml_path, 'r') as file:
        data = yaml.safe_load(file)
    return data['names']

In [3]:
# Create output directories for each class
def create_class_directories(classes, output_dir):
    for cls in classes:
        class_dir = os.path.join(output_dir, cls)
        os.makedirs(class_dir, exist_ok=True)

In [4]:
# Parse YOLO annotation file
def parse_annotation_file(annotation_file):
    with open(annotation_file, 'r') as file:
        lines = file.readlines()
    annotations = []
    for line in lines:
        line = line.strip().split()
        class_id = line[0]
        bbox = [float(x) for x in line[1:]]
        annotations.append((class_id, bbox))
    return annotations

In [10]:
def extract_and_save(image_path, annotations, classes, output_dir):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image not found or failed to load: {image_path}")
        return
    
    h, w, _ = image.shape
    
    for class_id, bbox in annotations:
        class_name = classes[int(class_id)]
        
        # YOLO format: (center_x, center_y, width, height) - normalized
        center_x, center_y, box_w, box_h = bbox
        x1 = max(0, int((center_x - box_w / 2) * w))
        y1 = max(0, int((center_y - box_h / 2) * h))
        x2 = min(w, int((center_x + box_w / 2) * w))
        y2 = min(h, int((center_y + box_h / 2) * h))
        
        # Crop image
        cropped_img = image[y1:y2, x1:x2]
        
        # Check if the cropped image is empty
        if cropped_img.size == 0:
            print(f"Empty crop for class {class_name} in image {image_path}. Skipping...")
            continue
        
        # Save cropped image to corresponding class folder
        class_folder = os.path.join(output_dir, class_name)
        file_name = os.path.basename(image_path)
        output_path = os.path.join(class_folder, file_name)
        
        # Save the cropped image
        cv2.imwrite(output_path, cropped_img)


In [13]:
# Main processing function
def process_dataset(images_dir, annotations_dir, classes, output_dir):
    create_class_directories(classes, output_dir)
    
    for annotation_file in tqdm(os.listdir(annotations_dir)):
        if annotation_file.endswith('.txt'):
            image_file = annotation_file.replace('.txt', '.jpg')  # adjust if your images are in a different format
            image_path = os.path.join(images_dir, image_file)
            annotation_path = os.path.join(annotations_dir, annotation_file)
            
            annotations = parse_annotation_file(annotation_path)
            extract_and_save(image_path, annotations, classes, output_dir)

# Train dataset preparation

In [7]:
# Set paths to your image and annotation directories
images_dir = r'Dataset-10/train/images'
annotations_dir = r'Dataset-10/train/labels'
output_dir = r'classification_dataset/train'
yaml_path = r'Dataset-10/data.yaml'

In [14]:
# Load classes from YAML file
classes = load_classes_from_yaml(yaml_path)

# Run the script
process_dataset(images_dir, annotations_dir, classes, output_dir)

  6%|▋         | 333/5219 [01:25<02:59, 27.28it/s]

Empty crop for class Building in image Dataset-10/train/images\D_1000221_png.rf.99046a5d120056f0ab9008bc2a10ca36.jpg. Skipping...


100%|██████████| 5219/5219 [06:51<00:00, 12.68it/s]


# Validation dataset preparation

In [15]:
# Set paths to your image and annotation directories
images_dir = r'Dataset-10/valid/images'
annotations_dir = r'Dataset-10/valid/labels'
output_dir = r'classification_dataset/valid'
yaml_path = r'Dataset-10/data.yaml'

In [16]:
# Load classes from YAML file
classes = load_classes_from_yaml(yaml_path)

# Run the script
process_dataset(images_dir, annotations_dir, classes, output_dir)

100%|██████████| 1281/1281 [02:32<00:00,  8.41it/s]
